## Working with Users

In [ ]:
import os
import tempfile
import requests
from arcgis.gis import GIS

## The `UserManager`

The manager allows for controlling of users

- Items
- Profile
- Groups
- Folders


### Working with Yourself!

- the easiest way to gain access to yourself is to access the `me` property
- this requires an authenticated `GIS` user

In [ ]:
gis = GIS(profile='your_online_profile')

In [ ]:
me = gis.users.me
me

#### `me` is a `User` Class

- The `User` object provides all the information you need to know about the account
    + access to license type
    + roles
    + content
    + folder
    + groups owned by you
    + items owned by you

#### Create a Folder

Much like an operating system, `ArcGIS Online and Enterprise` can store content in folders

In [ ]:
me.folders

In [ ]:
folder = gis.content.create_folder('museums')
folder

#### Add Content to the User's Folder

In [ ]:
url = "https://data.cityofnewyork.us/api/views/fn6f-htvy/rows.csv?accessType=DOWNLOAD"
with tempfile.TemporaryDirectory() as tmpdir:
    fp = os.path.join(tmpdir, 'nyc_musaemt2.csv')
    with requests.get(url, stream=True, verify=False) as r:
        r.raise_for_status()
        with open(fp, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
    ## Add the content!
    item = gis.content.add(item_properties={
            'title' : "NYC Museums Data",
            'type' : "CSV",
            'tags' : "Open Data, CSV, Museum, NYC, New York, New York City",
            'description' : "Locations of New York City Museums"
        }, 
        data=fp,
        folder='museums')

In [ ]:
item

#### Publish The `Museums`

In [ ]:
pitem = item.publish(publish_parameters={'name' : "NYC_Museums"}, address_fields={
    "Address": "ADRESS1",
    "Address2": "ADDRESS2",
    "City": "CITY",
    "Postal": "ZIP"
  })
pitem

#### Track User Item's Create and Modified Time

In [ ]:
import datetime
items = {}
folders = [fld['title'] for fld in me.folders] + [None]
for folder in folders:
    for item in me.items(folder):
        created_date = datetime.datetime.fromtimestamp(item.created/1000)
        mod_date = datetime.datetime.fromtimestamp(item.modified/1000)
        items[item.title] = {'item' : item, 'created' : created_date, 'modified' : mod_date}
        
items

#### Update User Profile

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    url = "https://img.favpng.com/13/0/13/cat-computer-icons-user-profile-avatar-png-favpng-0aXfSAjB7FwDVpeuUDXvWRLzd.jpg"
    r = requests.get(url)
    with open("./cat_profile.jpg", 'wb') as writer:
        writer.write(r.content)
    me.update(thumbnail="./cat_profile.jpg")
me